<a href="https://www.nvidia.com/dli"> <img src="../images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>

# 3.0 명명된 엔터티 인식기 구축
### (NVIDIA NeMo v1.0)

이 노트북에서는 의학 질병 초록에서 질병명을 찾는 애플리케이션을 구축합니다. 이 모델은 목록에서 이름을 "검색"하기보다는 언어의 맥락에서 특정 단어가 질병 참조임을 "인식"합니다.

**[3.1 커맨드 라인에서 토큰 분류](#3.1-커맨드-라인에서-토큰-분류)**<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.1.1 데이터 입력](#3.1.1-데이터-입력)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[3.1.1.1 IOB 태깅](#3.1.1.1-IOB-태깅)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.1.2 환경 구성 파일](#3.1.2-환경-구성-파일)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.1.3 Hydra-Enabled Python 스크립트](#3.1.3-Hydra-Enabled-Python-스크립트)<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.1.4 예제: 모델 트레이닝](#3.1.4-예제:-모델-트레이닝)<br>
**[3.2 도메인별 트레이닝](#3.2-도메인별-트레이닝)**<br>
&nbsp;&nbsp;&nbsp;&nbsp;[3.2.1 TensorBoard로 결과 시각화하기](#3.2.1-TensorBoard로-결과-시각화하기)<br>
**[3.3 평가](#3.3-평가)**<br>
**[3.4 추론](#3.4-추론)**<br>

NER 태스크의 경우, 텍스트 분류와 동일한 기본 단계에 따라 프로젝트를 구축하고 트레이닝 후 테스트 합니다. 그러나 이번에는, *도메인에 특정된*  BioMegatron 언어 모델에 있는 분류자를 트레이닝 할 예정입니다. BioMegatron은 대용량의 생의학 언어 코퍼스 ([PubMed](https://pubmed.ncbi.nlm.nih.gov/) abstracts와 전체 텍스트 상업적 사용 컬렉션) 에서 사전 훈련된 [BERT](https://arxiv.org/abs/1810.04805)-like [Megatron-LM](https://arxiv.org/pdf/1909.08053.pdf) 모델입니다.  우리는 질병 데이터 세트가 같은 생의학 영역에서 나왔기 때문에 일반 언어 모델과 비교하여 BioMegatron 에서 더 나은 성능을 기대할 수 있습니다. 

BioMegatron에 몇 가지 대안이 있으며 가장 눈에  띄는 모델은 [BioBERT](https://arxiv.org/abs/1901.08746)입니다. BioBERT와 비교하여 BioMegatron은 모델 사이즈에서 더 크며 더 큰 텍스트 코퍼스(말뭉치)에 대해 사전 훈련되어 있습니다. 

---
# 3.1 커맨드 라인에서 토큰 분류
저희가 이번 실습에서 답변하고자 하는 질문은 다음과 같습니다.: 

**의학적 Abstract에서 주어진 문장에는 어떤 질병이 언급되었나요?**<br>

NeMo에서 사용 가능한 NLP 모델을 상기시켜 봅니다. :

In [ ]:
!tree nemo/examples/nlp -L 1

우리는 NER을 위해 [토큰 분류](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/nlp/token_classification.html) 모델을 활용합니다. 왜냐하면 우리는 질병과 연관되는 단어를 분류하기 위해 "token(토큰)" 레벨에서 분류가 필요하기 때문입니다. 

## 3.1.1 데이터 입력
[1.0 데이터 살펴보기](010_ExploreData.ipynb) 노트북에서 확인한 것과 같이, NER 프로젝트를 위한 데이터세트는 질병 이름을 위해 IOB 태깅이 포함된 문장으로 구성딥니다. 여기서 문장의 각 단어는 명명된 엔티티의 내부(inside), 외부(outside), 시작(beginning)으로 태그가 지정됩니다. 

트레이닝 텍스트와 레이블 파일로는 각각 `text_train.txt`, `labels_train.txt`이 있습니다. 유효성 검증과 테스트 파일은 비슷한 이름 패턴을 따릅니다. 데이터 파일의 위치를 확인해 봅니다. 

In [ ]:
NER3_DATA_DIR = '/dli/task/data/NCBI_ner-3'
!ls -lh $NER3_DATA_DIR

In [ ]:
# Take a look at the data
print("*****\ntext_test.txt sample\n*****")
!head -n 3 $NER3_DATA_DIR/text_test.txt
print("\n*****\nlabels_test.txt sample\n*****")
!head -n 3 $NER3_DATA_DIR/labels_test.txt

### 3.1.1.1 IOB 태깅
NER 데이터 세트의 문장과 레이블은 _inside, outside, beginning (IOB)_ 태깅을 사용하여 서로 매핑됩니다. 이 매커니즘은 일반적으로 복수의 이름으로 명명된 엔티티 유형에도 사용할 수 있습니다. :
* B-{CHUNK_TYPE} – Beginning 청크 내에 있는 단어
* I-{CHUNK_TYPE} – 청크 내부(Inside)에 있는 단어들 
* O – 모든 청크의 외부(Outside)에 있음

이번 실습의 경우, 우리는 엔티티 (또는 청크) 유형으로의 "질병"만 찾고 있으므로, 우리는 다음의 3가지 클래스 이외는 식별할 필요가 없습니다. : I, O, B<br>
**세 개의 클래스**
* B - 질병 이름의 시작
* I - 질병 이름 내부에 있는 단어
* O - 모든 질병명을 제외

```text
Identification of APC2 , a homologue of the adenomatous polyposis coli tumour suppressor .
O              O  O    O O O         O  O   B           I         I    I      O          O  
```

다음은 `labels.csv` 파일에 정의되어 있습니다.:

In [ ]:
!head $NER3_DATA_DIR/label_ids.csv

우리가 언어 부분 분석에서 명사와 동사와 같은 2 종류의 명명된 엔티티를 찾는다면, 우리는 5 등급 IOB 체계를 사용할 것 입니다.:<br>
**다섯개 클래스**
* B-N - 명사 단어나 구문의 시작
* I-N - 명사 단어나 구문 내부 
* B-V - 동사 단어나 구문 시작
* I-V - 동사 단어나 구문 내부
* O   - 모든 명사나 동사를 제외

해당 주제에 대해 더 자세히 알아보려면 [다음 문서](http://cs229.stanford.edu/proj2005/KrishnanGanapathy-NamedEntityRecognition.pdf)를 확인해 보세요.

NCBI_ner-3 질병 데이터는 다음 [문서](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/nlp/token_classification.html#data-input-for-token-classification-model)의 설명에 따라 토큰 분류를 위해 올바른 유형으로 되어 있으므로 우리는 다음 환경 구성 파일을 확인해볼 준비가 되었습니다. 

## 3.1.2 환경 구성 파일
토큰 분류 디렉터리에 대한 자세한 내용은 다음을 참조하십시오.:

In [ ]:
TC_DIR = "/dli/task/nemo/examples/nlp/token_classification"
!tree $TC_DIR

NER를 위한 환경 구성 파일인  `token_classification_config.yaml`은 파일 위치, 사전 훈련 모델, 하이퍼 파라미터와 같은 모델, 트레이닝, 실험 관리 세부 정보를 지정합니다. 이 패턴은 기존에 텍스트 분류 구성 파일에서 사용되는 것과 동일한 패턴입니다. 우리는 텍스트 분류 프로젝트에서 소개되었던 `OmegaConf` 도구를 사용하여 각 섹션의 세부 사항을 살펴보겠습니다. 

In [ ]:
from omegaconf import OmegaConf

CONFIG_DIR = "/dli/task/nemo/examples/nlp/token_classification/conf"
CONFIG_FILE = "token_classification_config.yaml"

config = OmegaConf.load(CONFIG_DIR + "/" + CONFIG_FILE)

# print the model section
print(OmegaConf.to_yaml(config.model))

In [ ]:
# complete list of supported BERT-like models
from nemo.collections import nlp as nemo_nlp
nemo_nlp.modules.get_pretrained_lm_models_list()

`model`섹션에서는 모든 데이터 파일을 포함하는 `dataset.data_dir` 경로가 필요합니다. 현재 사용하고 있는 실제 파일 이름은 이미 기본 값을 만족하고 있으므로 재정의 할 필요는 없습니다. 

첫 번째 시도에서는, 우리는 `language_model.pretrained_model_name`를  `bert-base-cased`로 재정의함으로서 다른 실험에서 도메인 특성을 가지고 있는`biomegatron-bert-345m-cased`와 결과 값을 비교할 수 있습니다. 우리는 BioMegatron을 실행하기 위해 메모리 공간을 남겨야 하므로, 우리는 `dataset.max_seq_length`와 `batch_size` 값을 줄이도록 하겠습니다.

In [ ]:
# print the trainer section
print(OmegaConf.to_yaml(config.trainer))

효율성을 위해 `amp_level`을 'O1'로 설정할 수 있습니다. 우리가 비교하고자 하는 모델들이 크고 실행하는 데 오랜 시간이 소요되기 때문에 우리는 `max_epochs` 값을 작은 값으로 재정의할 것입니다. 

In [ ]:
# print the experiment manager section
print(OmegaConf.to_yaml(config.exp_manager))

지금은 `exp_manger` 기본 설정을 변경할 필요가 없습니다. 

## 3.1.3 Hydra-Enabled Python 스크립트
파이썬 스크립트인 `token_classification_train.py`와 `token_evaluate.py`는 환경 구성 파일에 정의된 토큰 분류 실험을 실행하기 위해 필요한 모든 것을 캡슐화 합니다. 이 경우, 트레이닝과 평가는 별도로 수행될 것으로 예상됩니다. 텍스트 분류와 마찬가지로 두 스크립트 모두 Facebook의 [Hydra](https://hydra.cc/) 도구를 구성 관리를 위해 활용함으로서 필요에 따라 구성 파일 값을 재정의함으로서 전체 실험을 커맨드 라인에서 실행할 수 있습니다. 

다시 설명하면, 우리가 변경하거나 재정의해야하는 파라미터 값은 다음과 같습니다.:

* `model.language_model.pretrained_model_name`: 'bert-base-cased'
* `model.dataset.data_dir`:  /dli/task/data/NCBI_ner-3
* `model.dataset.max_seq_length`: 64
* `model.train_ds.batch_size`: 32
* `model.val_ds.batch_size`:  32
* `model.test_ds.batch_size`:  32
* `trainer.amp_level`:  "O1"
* `trainer.max_epochs`:  3

## 3.1.4 예제: 모델 트레이닝

텍스트 분류 노트북에 있었던 유사한 실험을 실행한 것과 같이 `token_classification_train.py` 트레이닝 스크립트를 실행합니다. 

재정의를 위한 새로운 값들은 아래 셀에 제공됩니다. 적절한 재정의와 함께 명령문을 추가하고 셀을 실행합니다. 실습 수행에 어려움이 있는 경우, [솔루션](solutions/ex3.1.4.ipynb) 을 참고하세요.

In [ ]:
%%time
# The training takes about 2 minutes to run
   
TOKEN_DIR = "/dli/task/nemo/examples/nlp/token_classification"

# set the values we want to override
PRETRAINED_MODEL_NAME = 'bert-base-cased'
DATA_DIR = '/dli/task/data/NCBI_ner-3'
MAX_SEQ_LENGTH = 64
BATCH_SIZE = 32
AMP_LEVEL = 'O1'
MAX_EPOCHS = 3

# Override the config values in the command line
# FIXME

결과값은 어땠나요? 로그에 다음과 같은 항목이 포함되어 있을 것입니다.

```
    label                                                precision    recall       f1           support   
    O (label_id: 0)                                         99.34      99.35      99.34      21648
    B (label_id: 1)                                         85.86      89.21      87.50        769
    I (label_id: 2)                                         91.74      89.00      90.35       1073
    -------------------
    micro avg                                               98.54      98.54      98.54      23490
    macro avg                                               92.31      92.52      92.40      23490
    weighted avg                                            98.55      98.54      98.55      23490
    
Epoch 2: 100%|█| 199/199 [00:15<00:00, 12.45it/s, loss=0.0251, v_num=4-43, val_l
```

---
# 3.2 도메인별 트레이닝

다른 실험을 시도해 봅니다. 이번에는 `model.language_model.pretrained_model_name`를 `biomegatron-bert-345m-cased`로 재정의합니다. 이 모델은 3억 4,500만 개의 파라미터를 가진 대형 모델입니다. 따라서 실행 하는 데 시간이 오래 걸립니다. 

In [ ]:
%%time
# The training takes about 5-6 minutes to run
   
TOKEN_DIR = "/dli/task/nemo/examples/nlp/token_classification"

# set the values we want to override
PRETRAINED_MODEL_NAME = 'biomegatron-bert-345m-cased'
DATA_DIR = '/dli/task/data/NCBI_ner-3'
MAX_SEQ_LENGTH = 64
BATCH_SIZE = 32
AMP_LEVEL = 'O1'
MAX_EPOCHS = 3

# Override the config values in the command line
!python $TOKEN_DIR/token_classification_train.py \
        model.language_model.pretrained_model_name=$PRETRAINED_MODEL_NAME \
        model.dataset.data_dir=$DATA_DIR \
        model.dataset.max_seq_length=$MAX_SEQ_LENGTH \
        model.train_ds.batch_size=$BATCH_SIZE \
        model.validation_ds.batch_size=$BATCH_SIZE \
        model.test_ds.batch_size=$BATCH_SIZE \
        trainer.amp_level=$AMP_LEVEL \
        trainer.max_epochs=$MAX_EPOCHS

## 3.2.1 TensorBoard로 결과 시각화하기
[experiment manager](https://docs.nvidia.com/deeplearning/nemo/user-guide/docs/en/stable/core/core.html?highlight=tensorboard#experiment-manager) 는 텐서보드로 볼 수 있는 결과들을 저장합니다. 다음 셀을 실행하여 인스턴스에 대한 텐서보드 링크를 만든 다음 링크를 클릭하여 브라우저의 탭에서 텐서보드를 엽니다. 

In [ ]:
%%js
const href = window.location.hostname +'/tensorboard/';
let a = document.createElement('a');
let link = document.createTextNode('Open Tensorboard!');
a.appendChild(link);
a.href = "http://" + href;
a.style.color = "navy"
a.target = "_blank"
element.append(a);

실행한 모델들의 결과를 비교하려면 "f1" 스케일러를 선택합니다. 여러분은 함께 실행한 모든 모델을 보실 수도 있고, 개별 모델을 선택하여 비교할 수 있습니다. 이번 예제 비교에서는 5개의 Epoch가 실행되었습니다. 주황색 실선은 `bert-base-cased` 모델의 결과를 보여주고 파란색 실선은 `biomegatron-bert-345m-cased` 모델의 결과를 보여줍니다. BioMegatron 모델은 질병 이름을 더 잘 식별할 수 있기 때문에 매우 빠르게 잘 동작합니다. 5개 epoch 이후에 여전히 약간 높은 f1을 가지고 있습니다. 여러분의 프로젝트를 위해 선택하는 모델은 메모리, 시간, 성능 요구사항의 제약 조건에 따라 달라질 수 있습니다. 여러분의 결과가 학습 알고리즘의 임의성으로 예시와 다를 수 있음을 안내드립니다. 

<img src="images/tensorboard_02.png" width=800px>

---
# 3.3 평가

In [ ]:
# Restart the kernel
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

테스트 세트를 통해 모델을 평가하려면 `.nemo` 훈련된 모델의 위치를 지정해야 합니다. 각 실험은 `nemo_experiments` 아래의 시간 표기한 디렉토리에 결과를 실행 합니다. 드릴 다운하면 최종 `token_classification_model.nemo` 가 있는 `checkpoints` 폴더를 찾을 수 있습니다. 다음 셀에서는 Python 로직의 일부를 사용하여 모델 목록을 캡쳐하고 최신 모델 목록을 식별해 봅니다. 

In [ ]:
import glob

nemo_model_paths = glob.glob('nemo_experiments/token_classification_model/*/checkpoints/*.nemo')

# Sort newest first
nemo_model_paths.sort(reverse=True)
print("The latest model is \n{}".format(nemo_model_paths[0]))

테스트 세트에 대해 평가를 진행하는 몇 가지 방법이 있습니다.:
1. `token_classification_evaluate.py`를 기존과 같은 재정의 값과 함께 실행하고 `pretrained_model`를 `.nemo` 형식 값으로 추가 재정의 합니다. 

```text
   !python $TOKEN_DIR/token_classification_evaluate.py \
        model.dataset.data_dir=$DATA_DIR \
        model.dataset.max_seq_length=$MAX_SEQ_LENGTH \
        model.train_ds.batch_size=$BATCH_SIZE \
        model.validation_ds.batch_size=$BATCH_SIZE \
        model.test_ds.batch_size=$BATCH_SIZE \
        model.language_model.pretrained_model_name=$PRETRAINED_MODEL_NAME \
        pretrained_model=$LATEST_MODEL
```
        
2. 트레이닝된 모델 체크포인트를 복원하여 모델을 인스턴스화 하고 NeMo 메소드를 실행하여 테스트 세트를 평가합니다.<br>
   우리는 이 방법으로 실습을 진행합니다. 

In [ ]:
# Instantiate the model by restoring from the .nemo checkpoint
from nemo.collections import nlp as nemo_nlp

LATEST_MODEL = nemo_model_paths[0]
model = nemo_nlp.models.TokenClassificationModel.restore_from(LATEST_MODEL)

`evaluate_from_file` 메소드를 사용하여 테스트 세트로 모델을 평가합니다.`add_confusion_matrix`를 True로 설정하여 모델이 얼마다 잘 했는지 시각적으로 표현합니다. .

In [ ]:
import os.path

DATA_DIR = '/dli/task/data/NCBI_ner-3'
OUTPUT_DIR = '/dli/task/nemo_experiments/token_classification_model/logs'
model.evaluate_from_file(
    text_file=os.path.join(DATA_DIR, 'text_test.txt'),
    labels_file=os.path.join(DATA_DIR, 'labels_test.txt'),
    output_dir=OUTPUT_DIR,
    add_confusion_matrix=True,
    normalize_confusion_matrix=True,
    batch_size=1
)

결과 값은 다음과 같이 표시 됩니다.

```
[NeMo I 2021-06-29 00:42:16 token_classification_model:499]                  precision    recall  f1-score   support
    
    O (label id: 0)     0.9958    0.9910    0.9934     22450
    B (label id: 1)     0.8886    0.9135    0.9009       960
    I (label id: 2)     0.8724    0.9374    0.9038      1087
    
           accuracy                         0.9856     24497
          macro avg     0.9189    0.9473    0.9327     24497
       weighted avg     0.9861    0.9856    0.9858     24497


최종 Confusion 매트릭트 시각화는 밝은 대각선을 보여주며, 이는 예측 레이블이 모든 레이블 유형 (IOB)에 대해 높은 정확도로 실제 레이블과 일치했음을 나타냅니다. 
```

<img src="images/ner_confusion_matrix.png">

---
# 3.4 추론
쿼리 목록에 대한 추론을 실행하려면, `add_predictions` 메소드와 함께 이미 로드된 동일한 모델을 사용하십시오 .

In [ ]:
queries = ["Clustering of missense mutations in the ataxia - telangiectasia gene in a sporadic T - cell leukaemia . ",
    "Ataxia - telangiectasia ( A - T ) is a recessive multi - system disorder caused by mutations in the ATM gene at 11q22 - q23 ( ref . 3 ) . ",
    "The risk of cancer , especially lymphoid neoplasias , is substantially elevated in A - T patients and has long been associated with chromosomal instability . ",
    "By analysing tumour DNA from patients with sporadic T - cell prolymphocytic leukaemia ( T - PLL ) , a rare clonal malignancy with similarities to a mature T - cell leukaemia seen in A - T , we demonstrate a high frequency of ATM mutations in T - PLL . ",
    "In marked contrast to the ATM mutation pattern in A - T , the most frequent nucleotide changes in this leukaemia were missense mutations . ",
    "These clustered in the region corresponding to the kinase domain , which is highly conserved in ATM - related proteins in mouse , yeast and Drosophila . ",
    "The resulting amino - acid substitutions are predicted to interfere with ATP binding or substrate recognition . ",
    "Two of seventeen mutated T - PLL samples had a previously reported A - T allele . ",
    "In contrast , no mutations were detected in the p53 gene , suggesting that this tumour suppressor is not frequently altered in this leukaemia . ",
    "Occasional missense mutations in ATM were also found in tumour DNA from patients with B - cell non - Hodgkins lymphomas ( B - NHL ) and a B - NHL cell line . "
          ]

In [ ]:
results = model.add_predictions(queries, output_file='predictions.txt')

In [ ]:
!cat predictions.txt

---
<h2 style="color:green;">축하합니다!</h2>

여러분은 NeMo를 마스터하고 다음을 배웠습니다:
* 명명된 엔티티 인식기를 구축하는 방법 
* 도메인별 모델에 적용하는 방법
* 체크포인트에서 쿼리를 사용하여 NER 모델을 테스트하는 방법

<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>